# Knowledge and Data 2017: 3rd practical assignment 
## RDF Data, RDFS knowledge and inferencing 

Your name: **Can (John) Lokman**

Your VUNetID: **CLN940** \ Student Number: **2563162**

If you do not provide your name and VUNetID we will not accept your submission. 

At the end of this exercise you should be able to 

1. access local an external data via SPARQL both from within a python programming environment and stand-alone with a GUI, such as YASGUI (www.yasgui.org), and this way integrate data from different sources  
2. Model your own first knowledge base, in this case an RDF Schema knowledge graph
3. Implement inference rules 

Follow this Notebook step-by-step. 

Of course, you can do the exercises in any Programming Editor of your liking. 
But you do not have to. Feel free to simply write code in the Notebook. When 
everythink is filled in and works, safe the Notebook and submit it 
as a Jupyter Notebook, i.e. with an ipynb extension. Please use as name of the 
Notebook your studentID+Assignment3.ipynb.  

Other than in courses dedicated to programming we will not evaluate the style
of the programs. But we will test your programs on other data than we provide, 
and your program should give the correct answers to those test-data as well. 

Before you start, install the Stardog Triplestore, and make sure the server is running (follow the explanation in the file tools.md that was provided in week 1 - https://github.com/kadevgraaf/semanticweb-web-application-tutorial/blob/master/tools.md#4-the-stardog-triple-store-version-4-or-higher). 

Then, add the file example-from-slides.ttl to a newly created database, say called assignment3. 

## Task 1: (2 points) Integrate local and external data

You can integrate SPARQL queries into your python code by using the SPARQLWrapper library as follows: 

In [52]:
#THIS ONLY WORKS WHEN YOU ARE ONLINE
from rdflib import Graph, RDF, Namespace, Literal, URIRef
from SPARQLWrapper import SPARQLWrapper, JSON

sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.setQuery("""
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    SELECT ?label
    WHERE { <http://dbpedia.org/resource/Amsterdam> rdfs:label ?label }
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
for result in results["results"]["bindings"]:
   print(result["label"]["value"])

# This code accesses the DBPedia database, and returns the result of the specified SPARQL query (test it!)

Amsterdam
أمستردام
Amsterdam
Ámsterdam
Amsterdam
Amsterdam
アムステルダム
Amsterdam
Amsterdam
Amesterdão
Амстердам
阿姆斯特丹


While the above is just an example code on how to use RDFLIb and SPARQLWRAPPER to access external/interenal SPARQL endpoints, and manipulate the results in Python. 

Your tasks is now to write a SPARQL query that extracts all the cities from your local knowledge graph (constructed by loading the file example-from-slides.ttl) and uses the results to find the number of inhabitants of these cities and the longitute and latitude information (if available) from DBPedia. 

Add those triples to your graph using both namespaces (e.g. ex:Amsterdam dbo:populationTotal "123456") and print in turtle.   

Import example-from-slides.ttl: 

In [53]:
from rdflib import Graph, RDF, Namespace, Literal, URIRef
g = Graph()

EX = Namespace('http://example.com/kad2017/')   # Define a namespace variable for Python environment...
g.bind('ex',EX)                                 # and bind it to the string 'ex' in graph environment.
# Usage:    EX[<Prefix here>]
# Example:
# EX["England"] # creates a new element "http://example.com/kad2017/England"
# This is equivalent to:
# URIRef("http://example.com/kad2017/England")


def serialize():
    print g.serialize(format='turtle')

def save(filename):
    with open(filename, 'w') as f:
        g.serialize(f, format='turtle')

def load(filename):
    with open(filename, 'r') as f:
        g.load(f, format='turtle')

load('example-from-slides.ttl')                 # If using Jupyter, please uncomment this line
#load('Assignment 3\\example-from-slides.ttl')  # If using PyCharm, please uncomment this line

serialize()

@prefix dbo: <http://dbpedia.org/ontology/> .
@prefix dbr: <http://dbpedia.org/resource/> .
@prefix ex: <http://example.com/kad2017/> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

ex:Netherlands a ex:Country ;
    ex:contains ex:Ijsselmeer ;
    ex:containsCity ex:Rotterdam ;
    ex:has_Capital ex:Amsterdam ;
    ex:has_Name "TheNetherlands" ;
    ex:neighbours ex:Belgium .

ex:hasCapital rdfs:range ex:Capital ;
    rdfs:subPropertyOf ex:containsCity .

ex:neighbours rdfs:subPropertyOf ex:closeBy .

ex:Amsterdam a ex:Capital ;
    ex:closeBy ex:Germany .

ex:Belgium a ex:Country .

ex:EuropeanCountry rdfs:subClassOf ex:Country .

ex:Germany a ex:EuropeanCountry ;
    ex:hasCapital ex:Berlin .

ex:closeBy rdfs:domain ex:Location ;
    rdfs:range ex:Location .

ex:containsCity rdfs:domain ex:Country ;
    rdfs:r

In [55]:
# The current state of the graph
for s,p,o in g:
    print s + "|||" + p + "|||" + o

http://example.com/kad2017/hasCapital|||http://www.w3.org/2000/01/rdf-schema#subPropertyOf|||http://example.com/kad2017/containsCity
http://example.com/kad2017/Netherlands|||http://example.com/kad2017/contains|||http://example.com/kad2017/Ijsselmeer
http://example.com/kad2017/hasCapital|||http://www.w3.org/2000/01/rdf-schema#range|||http://example.com/kad2017/Capital
http://example.com/kad2017/Country|||http://www.w3.org/2000/01/rdf-schema#subClassOf|||http://example.com/kad2017/Location
http://example.com/kad2017/closeBy|||http://www.w3.org/2000/01/rdf-schema#range|||http://example.com/kad2017/Location
http://example.com/kad2017/neighbours|||http://www.w3.org/2000/01/rdf-schema#subPropertyOf|||http://example.com/kad2017/closeBy
http://example.com/kad2017/Netherlands|||http://example.com/kad2017/containsCity|||http://example.com/kad2017/Rotterdam
http://example.com/kad2017/Netherlands|||http://example.com/kad2017/neighbours|||http://example.com/kad2017/Belgium
http://example.com/kad201

Get all relationships of 'ex:City':

In [56]:
import rdflib
from rdflib import Literal, URIRef

qres = g.query(
    
    """
    PREFIX ex: <http://example.com/kad2017/>

    SELECT ?x
        WHERE {
            {?x  ?anyPredicate ex:City .}
            UNION
            {ex:City  ?anyPredicate ?anyObject .}
        }
    """)

for row in qres:
    print("%s"  % row)

http://example.com/kad2017/Capital
http://example.com/kad2017/containsCity


Return all cities in the triple store:

In [57]:
qres = g.query(
    
    """
    PREFIX ex: <http://example.com/kad2017/>
    PREFIX dbp: <http://dbpedia.org/resource/>
    PREFIX dbo: <http://dbpedia.org/ontology/>

    SELECT ?dbpediaCity
        WHERE {
            # Get all cities in the triple store
            {?subject ex:containsCity ?x .}
            UNION
            {?x a ex:Capital .}
            UNION
            {?subject ex:hasCapital ?x .}
            
            # Replace 'http://example.com/kad2017/' with 'http://dbpedia.org/resource/' in the returned URIs
            BIND (URI(REPLACE(STR(?x), STR(ex:), STR(dbp:))) AS ?dbpediaCity)
            #BIND (URI(REPLACE(STR(?dbpediaCity), STR(dbp:), "a")) AS ?dbpediaCity)
        }
        
    """
)

dbpediaCities = []
for row in qres:
    #print("%s"  % row)
    dbpediaCities.append("%s"  % row)

from pprint import pprint
pprint (dbpediaCities)

[u'http://dbpedia.org/resource/Rotterdam',
 u'http://dbpedia.org/resource/Amsterdam',
 u'http://dbpedia.org/resource/Berlin']


***Return population information (required) and latitude/longitude of each city (optional):
(As Rotterdam does not have a population info, it is not among the returned results. Despite the missing population information, Rotterdam could have been included with the an optional "dbp:populationBlank" property, or by simply encapsulating the "<{val}> dbo:populationTotal ?pop ." query in OPTIONAL keyword, but this considerably complicates results, and makes their printing an even more complicated process.)***

In [58]:
final_results = []
for each_city in dbpediaCities: 
    sparql.setQuery(
        """
        
        PREFIX ex: <http://example.com/kad2017/>
        PREFIX dbr: <http://dbpedia.org/resource/>
        PREFIX dbo: <http://dbpedia.org/ontology/>
        PREFIX dbp: <http://dbpedia.org/property/>
        
        
        SELECT ?stringCity ?pop ?lat ?lon
        
        WHERE {{ 
            {{BIND (URI(REPLACE(STR(<{val}>), STR(dbp:), "")) AS ?stringCity)}}
            <{val}> dbo:populationTotal ?pop .
            OPTIONAL{{<{val}> dbp:latd ?lat}}
            OPTIONAL{{<{val}> dbp:longd ?lon}}
        }}
        
        """.format(val=each_city)
    )
    
    
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    for result in results["results"]["bindings"]:
        final_results.append([result])
        

for each_result in final_results:
    print (str(
        "CITY: " + each_result[0]["stringCity"]["value"]) + " , " 
           "POPULATION: " + each_result[0]["pop"]["value"]+ " , " 
           "LATITUDE: " + each_result[0]["lat"]["value"] + " , "
           "LONGITUDE: " + each_result[0]["lon"]["value"]
           )

CITY: http://dbpedia.org/resource/Amsterdam , POPULATION: 842343 , LATITUDE: 52 , LONGITUDE: 4
CITY: http://dbpedia.org/resource/Berlin , POPULATION: 3610156 , LATITUDE: 52 , LONGITUDE: 13


## Task 2: (3 points)  Implement basic inferencing rules 

In the lecture we showed that the RDFS inference rules can be used to infer new knowledge. For example, infer class membership based on rdfs:domain or infer relationships between subjects and objects based on rdfs:subPropertyOf.

Create rules (or 1 rule?!) to inference class membership based on the RDF Schema language features 
*	For example: infer that an instance belongs to a class because of domain and range restrictions
*	For example: infer that an instance belongs to a (super)class because it also belongs to a subclass

You should implement at least 4 of the following rules:  

*     (rdfs2) If G contains the triples (a rdfs:domain x.) and (u a y.)  then infer the triple (u rdf:type x.)
*     (rdfs3) If G contains the triples (a rdfs:range x.) and (u a v.) then infer the triple (v rdf:type x .)
*     (rdfs5) If G contains the triples (u rdfs:subPropertyOf v.) and (v rdfs:subPropertyOf x.) then infer the triple
(u rdfs:subPropertyOf x.) 
*     (rdfs7) If G contains the triples (a rdfs:subPropertyOf b.) and (u a y.) then infer the triple (u b y) 
*     (rdfs9) If G contains the triples (u rdfs:subClassOf x.) and (v rdf:type u.) then infer the triple
 (v rdf:type x.)   -> this one was not mentioned in the lecture, but is a very important one. 
*     (rdfs11) If G contains the triples (u rdfs:subClassOf v.) and (v rdfs:subClassOf x.) then infer the triple
(u rdfs:subClassOf x.)

Actually, please note that you will have already implemented rdfs as part of the assignment of Module 2. 

Run your rule reasoner on your knowledge graph.

In [59]:
# The current state of the triple store.
for s,p,o in g:
    print s + "     ---      " + p + "     ---      " + o

http://example.com/kad2017/hasCapital     ---      http://www.w3.org/2000/01/rdf-schema#subPropertyOf     ---      http://example.com/kad2017/containsCity
http://example.com/kad2017/Netherlands     ---      http://example.com/kad2017/contains     ---      http://example.com/kad2017/Ijsselmeer
http://example.com/kad2017/hasCapital     ---      http://www.w3.org/2000/01/rdf-schema#range     ---      http://example.com/kad2017/Capital
http://example.com/kad2017/Country     ---      http://www.w3.org/2000/01/rdf-schema#subClassOf     ---      http://example.com/kad2017/Location
http://example.com/kad2017/closeBy     ---      http://www.w3.org/2000/01/rdf-schema#range     ---      http://example.com/kad2017/Location
http://example.com/kad2017/neighbours     ---      http://www.w3.org/2000/01/rdf-schema#subPropertyOf     ---      http://example.com/kad2017/closeBy
http://example.com/kad2017/Netherlands     ---      http://example.com/kad2017/containsCity     ---      http://example.com/kad20

***(rdfs11)***


If G contains the triples (x rdfs:subClassOf y.) and (y rdfs:subClassOf z.) then infer the triple (x rdfs:subClassOf z.):

In [60]:
qres = g.query(
    
    """
    PREFIX ex: <http://example.com/kad2017/>
    PREFIX dbp: <http://dbpedia.org/resource/>
    PREFIX dbo: <http://dbpedia.org/ontology/>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

    CONSTRUCT {?x rdfs:subClassOf ?z .}
    WHERE
        {
            ?x rdfs:subClassOf ?y .
            ?y rdfs:subClassOf ?z . 
    }
    
    """
)

print ("The following triples are inferred:")
for row in qres:
    print(" %s         %s             %s"  % row)

The following triples are inferred:
 http://example.com/kad2017/EuropeanCountry         http://www.w3.org/2000/01/rdf-schema#subClassOf             http://example.com/kad2017/Location
 http://example.com/kad2017/Capital         http://www.w3.org/2000/01/rdf-schema#subClassOf             http://example.com/kad2017/Location


***(rdfs5)***

If G contains the triples (p rdfs:subPropertyOf q.) and (q rdfs:subPropertyOf r.) then infer the triple (p rdfs:subPropertyOf r.)

In [61]:
qres = g.query(
    
    """
    PREFIX ex: <http://example.com/kad2017/>
    PREFIX dbp: <http://dbpedia.org/resource/>
    PREFIX dbo: <http://dbpedia.org/ontology/>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

    CONSTRUCT {?p rdfs:subPropertyOf ?r .}
    WHERE
        {
            ?p rdfs:subPropertyOf ?q .
            ?q rdfs:subPropertyOf ?r . 
    }
    
    """
)

print ("The following triples are inferred:")
for row in qres:
    print(" %s         %s             %s"  % row)

The following triples are inferred:
 http://example.com/kad2017/hasCapital         http://www.w3.org/2000/01/rdf-schema#subPropertyOf             http://example.com/kad2017/contains


***(rdfs7)***

If G contains the triples (p rdfs:subPropertyOf q.) and (x p y.) then infer the triple (x q y):

In [62]:
qres = g.query(
    
    """
    PREFIX ex: <http://example.com/kad2017/>
    PREFIX dbp: <http://dbpedia.org/resource/>
    PREFIX dbo: <http://dbpedia.org/ontology/>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

    CONSTRUCT {?x ?q ?y .}
    WHERE
        {
            ?p rdfs:subPropertyOf ?q .
            ?x ?p ?y .
            
    }
    
    """
)

print ("The following triples are inferred:")
for row in qres:
    print(" %s         %s             %s"  % row)

The following triples are inferred:
 http://example.com/kad2017/Germany         http://example.com/kad2017/containsCity             http://example.com/kad2017/Berlin
 http://example.com/kad2017/Netherlands         http://example.com/kad2017/closeBy             http://example.com/kad2017/Belgium
 http://example.com/kad2017/Netherlands         http://example.com/kad2017/contains             http://example.com/kad2017/Rotterdam


***(rdfs3)***

If G contains the triples (p rdfs:range x.) and (y p z.) then infer the triple (z rdf:type x .):


In [63]:
qres = g.query(
    
    """
    PREFIX ex: <http://example.com/kad2017/>
    PREFIX dbp: <http://dbpedia.org/resource/>
    PREFIX dbo: <http://dbpedia.org/ontology/>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>


    CONSTRUCT {?z rdf:type ?x .}
    WHERE
        {
            ?p rdfs:range ?x .
            ?y ?p ?z .
    }
    
    """
)

print ("The following triples are inferred:")
for row in qres:
    print(" %s         %s             %s"  % row)

The following triples are inferred:
 http://example.com/kad2017/Germany         http://www.w3.org/1999/02/22-rdf-syntax-ns#type             http://example.com/kad2017/Location
 http://example.com/kad2017/Rotterdam         http://www.w3.org/1999/02/22-rdf-syntax-ns#type             http://example.com/kad2017/City
 http://example.com/kad2017/Berlin         http://www.w3.org/1999/02/22-rdf-syntax-ns#type             http://example.com/kad2017/Capital


## Task 3: (3 points) Build your very own RDFS knowledge graph. 


Define a small RDF Schema vocabulary in Turtle. You can choose your own domain.
*	The schema should define at least 4 classes, 4 properties, and 4 instances.
*   The properties should be used to relate the instances
*	The instances should be a member of your classes
*	All resources should have an rdfs:label in a suitable language.

You should use (at least) the following language features of RDF and RDFS:

* 	rdf:type (or 'a')
* 	rdfs:subClassOf
* 	rdfs:subPropertyOf
* 	rdfs:domain and rdfs:range
*	rdfs:label

Be sure to define the 'rdf:' and 'rdfs:' namespace prefixes for RDF and RDF Schema in your file (perhaps have a look at http://prefix.cc)

You can either use a text editor, use Snapper (http://jiemakel.github.io/snapper, which is YASGUI-based). You can also  add the axioms directly (programatically) to your Knowledge Graph as you did last week. Play around with the inference rules you have created in the previous task to make sure that you some added some implicit knowledge, that becomes "visible" via inferencing. (this will be useful for the next task). 

Create an RDFLIB graph datastructure *myRDFSgraph* and add your knowledge to the graph. 

***The Turtle file created using snapper can is copy-pasted below (it is also saves as a .ttl file, which is parsed in the cell afterwards):***

In [ ]:
@prefix ex: <http://example.org/> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .


# CLASSES
ex:aparment a rdfs:class ;
                rdfs:label "apartment"@en ;
                rdfs:subClassOf ex:residentialUnit .

ex:unit a rdfs:class ;
                rdfs:label "unit"@en .

ex:residentialUnit a rdfs:class ;
                rdfs:label "residential unit"@en ;
                rdfs:subClassOf ex:unit .

ex:commercialUnit a rdfs:class ;
                rdfs:label "commercial unit"@en ;
                rdfs:subClassOf ex:unit .

ex:person a rdfs:class ;
                rdfs:label "person"@en .

ex:child a rdfs:class ;
                rdfs:label "child"@en ;
                rdfs:subClassOf ex:person .

ex:adult a rdfs:class ;
                rdfs:label "adult"@en ;
                rdfs:subClassOf ex:person .

ex:company a rdfs:class ;
        rdfs:label "company" .


# PROPERTIES

ex:residesAt a rdfs:property ;
                rdfs:label "resides at"@en ;
                rdfs:domain ex:person ;
                rdfs:range ex:residentialUnit .

ex:rents a rdfs:property ;
                rdfs:label "rents"@en ;
                rdfs:subPropertyOf ex:residesAt .

ex:childOf a rdfs:property ;
                rdfs:label "child of"@en ;
                rdfs:subPropertyOf ex:residesWith .


ex:parentOf a rdfs:property ;
                rdfs:label "parent of"@en .

ex:worksAt a rdfs:property ;
                rdfs:label "works at"@en ;
                rdfs:domain ex:person ;
                rdfs:range ex:commercialUnit .

ex:hasOfficeAt a rdfs:property ;
                rdfs:label "has office at"@en ;
                rdfs:domain ex:company ;
                rdfs:range ex:commercialUnit ;
                rdfs:subPropertyOf ex:leases .

ex:leases a rdfs:property ;
                rdfs:label "leases"@en ;
                rdfs:domain ex:company ;
                rdfs:range ex:commercialUnit .




# INSTANCES
ex:John a ex:person ;
                rdfs:label "John"@en ;
                ex:hasChild ex:Grace ;
                ex:worksAt ex:Clokman .
                

ex:Grace a ex:child ;
                rdfs:label "Grace"@en ;
                ex:childOf ex:John.

ex:BaronWayApartment a ex:apartment ;
                rdfs:label "Baron Way Apartment"@en .

ex:ZuidOffice a ex:commercialUnit ;
                rdfs:label "Commercial Unit"@en .



# RELATIONSHIPS
ex:apartment rdfs:subClassOf ex:residentialUnit .
ex:residentialUnit rdfs:subClassOf ex:unit .

ex:rents rdfs:subPropertyOf ex:residesAt .
ex:rents rdfs:domain ex:person .
ex:rents rdfs:range ex:unit .

ex:John ex:rents ex:BaronWayApartment .

ex:Clokman ex:leases ex:ZuidOffice .

Serialise your RDF graph *myRDFSgraph* in Turtle, so that we can check your "design".

***Data structure is created and serialized as below:***

In [64]:
from rdflib import Graph, RDF, Namespace, Literal, URIRef
myRDFSgraph = Graph()

EX = Namespace('http://example.com/kad2017/')   # Define a namespace variable for Python environment...
myRDFSgraph.bind('ex',EX)                                 # and bind it to the string 'ex' in graph environment.
# Usage:    EX[<Prefix here>]
# Example:
# EX["England"] # creates a new element "http://example.com/kad2017/England"
# This is equivalent to:
# URIRef("http://example.com/kad2017/England")


def serialize():
    print myRDFSgraph.serialize(format='turtle')

def save(filename):
    with open(filename, 'w') as f:
        myRDFSgraph.serialize(f, format='turtle')

def load(filename):
    with open(filename, 'r') as f:
        myRDFSgraph.load(f, format='turtle')

load('example-from-slides.ttl')     # If using Jupyter, please uncomment this line
#load('Assignment 3\\john-life.ttl')  # If using PyCharm, please uncomment this line

serialize()

@prefix ex: <http://example.com/kad2017/> .
@prefix ex1: <http://example.org/> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

ex1:adult a rdfs:class ;
    rdfs:label "adult"@en ;
    rdfs:subClassOf ex1:person .

ex1:aparment a rdfs:class ;
    rdfs:label "apartment"@en ;
    rdfs:subClassOf ex1:residentialUnit .

ex1:childOf a rdfs:property ;
    rdfs:label "child of"@en ;
    rdfs:subPropertyOf ex1:residesWith .

ex1:hasOfficeAt a rdfs:property ;
    rdfs:label "has office at"@en ;
    rdfs:domain ex1:company ;
    rdfs:range ex1:commercialUnit ;
    rdfs:subPropertyOf ex1:leases .

ex1:parentOf a rdfs:property ;
    rdfs:label "parent of"@en .

ex1:rents a rdfs:property ;
    rdfs:label "rents"@en ;
    rdfs:domain ex1:person ;
    rdfs:range ex1:unit ;
    rdfs:subPropertyOf ex1:residesAt .

ex1:worksAt a r

### Task 4 (2 points) Compare local inferences with Stardog results

Upload your own RDF Schema to Stardog (go to http://localhost:5820 but make sure you have it running first!), and  access it using http://localhost:5820

Try to formulate a SPARQL query that queries your schema and some of its instances. Make sure that you write "interesting" queries that allow you to see a difference between the query results with inferencing and without inferencing. 

Try the query with and without inferencing (you can easily switch inferencing on and off if you use the Stardog query interface)

Tip: first write and test your query using Legacy YASGUI (http://legacy.yasgui.org) and the SPARQL endpoint of your own Stardog triplestore (http://localhost:5820/YOURDATABASENAME/query). Then copy the query to the Stardog-provided query interface, to test it with inferencing.

Provide the query in the Jupyter Notebook


***Query 1:***

In [ ]:
# When reasoner is on, this query returns where John resides at, and it does not when the reasoner is off (or on YASGUI). 'residesAt' is inferred from the 'rents' property and its subProperty relationship to 'residesAt') 

PREFIX ex: <http://example.org/>

SELECT ?o
WHERE { 
  ex:John ex:residesAt ?o
}

***Query 2:***

In [ ]:
# When reasoner is off/on YASGUI, this query returns only one binding (ex:apartment). While it is on (on Stardog), returns four results regarding (super)classes of BaronWayApartment. The reasoning is based on class relationships (weak inference). 

PREFIX ex: <http://example.org/>

SELECT ?o
WHERE { 
  ex:BaronWayApartment a ?o
}

***Query 3:***

In [ ]:
# When reasoner is off, this query returns only the fact that 'Clokman' leases 'ZuidOffice'. However, when the reasoner is on, it is also returned by way of inference that Clokman is a company. This is inferred from range and domain restrictions of the 'leases' property. 

PREFIX ex: <http://example.org/>

SELECT ?p ?o
WHERE { 
  ex:Clokman ?p ?o
}

***Query 4:***

In [ ]:
# When reasoner is off, this query returns only the facts that Grace is a child of John and she is a 'child'. When the reasoner is on, however, it returns that facts that she resides with John (inferred from 'childOf' is a subproperty of 'residesWith' [in a simplistic scenario]), and is a 'person' (inferred from 'child' being a subclass of 'person').
PREFIX ex: <http://example.org/>

SELECT ?p ?o
WHERE { 
  ex:Grace ?p ?o
}